In [1]:
##### Check GPU #####
import torch

num_gpus = torch.cuda.device_count()
if num_gpus > 0:
    print(f"There is {num_gpus} GPU in the system")
else:
    print("No GPU in the system")

/eos/user/l/lcondren/miniconda3/envs/exatrkx-gpu2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


There is 1 GPU in the system


In [4]:
######## Test: R & Hit_id modulewise_true_edge compare #######

import torch
import csv
import os
import numpy as np
from bokeh.io import output_notebook, show, export_png
from bokeh.plotting import figure, row
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource
from bokeh.palettes import viridis
from bokeh.models.annotations import Label
output_notebook()

event_id = 0
event_id_2 =1

#file_path_1 = f"./datasets/Lambda500_pre_selection_mix_hit/feature_store/QuirkTracking_sample/{event_id}"
file_path_1 = f"output/test"
file_path_2 = f"output/test"

# Check if both files exist
if os.path.exists(file_path_1) and os.path.exists(file_path_2):
    sample_data_1 = torch.load(file_path_1, map_location="cpu")
    sample_data_2 = torch.load(file_path_2, map_location="cpu")
else:
    raise FileNotFoundError(f"File not found")

num_tracks = 100

# Create the first plot
p1 = figure(title='Truth graph ("R" processing)', x_axis_label='x', y_axis_label='y', height=400, width=400)
r, phi, z = sample_data_1.cpu().x.T
true_edges = sample_data_1.modulewise_true_edges
true_unique, true_lengths = sample_data_1.pid[true_edges[0]].unique(return_counts=True)
pid = sample_data_1.pid
x = r * np.cos(phi)
y = r * np.sin(phi)
cmap = viridis(num_tracks)
source = ColumnDataSource(dict(x=x.numpy(), y=y.numpy()))
p1.circle(x='x', y='y', source=source, color=cmap[0], size=1, alpha=0.1)
for i, track in enumerate(true_unique[true_lengths >= 5][:num_tracks]):
    track_true_edges = true_edges[:, pid[true_edges[0]] == track]
    X_edges, Y_edges = x[track_true_edges].numpy(), y[track_true_edges].numpy()
    X = np.concatenate(X_edges)
    Y = np.concatenate(Y_edges)
    p1.circle(X, Y, color=cmap[i], size=5)
    p1.multi_line(X_edges.T.tolist(), Y_edges.T.tolist(), color=cmap[i])

# Create the second plot
p2 = figure(title='Truth graph ("Hit_id" processing)', x_axis_label='x', y_axis_label='y', height=400, width=400)
r, phi, z = sample_data_2.cpu().x.T
true_edges = sample_data_2.modulewise_true_edges
true_unique, true_lengths = sample_data_2.pid[true_edges[0]].unique(return_counts=True)
pid = sample_data_2.pid
x = r * np.cos(phi)
y = r * np.sin(phi)
cmap = viridis(num_tracks)
source = ColumnDataSource(dict(x=x.numpy(), y=y.numpy()))
p2.circle(x='x', y='y', source=source, color=cmap[0], size=1, alpha=0.1)
for i, track in enumerate(true_unique[true_lengths >= 5][:num_tracks]):
    track_true_edges = true_edges[:, pid[true_edges[0]] == track]
    X_edges, Y_edges = x[track_true_edges].numpy(), y[track_true_edges].numpy()
    X = np.concatenate(X_edges)
    Y = np.concatenate(Y_edges)
    p2.circle(X, Y, color=cmap[i], size=5)
    p2.multi_line(X_edges.T.tolist(), Y_edges.T.tolist(), color=cmap[i])

# Combine both plots into one layout
combined_layout = gridplot([[p1, p2]])

# Show the combined layout
show(combined_layout)

# Export the combined layout as PNG
#export_png(combined_layout, filename=f"./plot/try/{event_id}_combined.png")

Loading BokehJS ...

UnpicklingError: could not find MARK

In [3]:
##### Draw training performance plot (loss purity eff for metric and GNN) #####

import os
import pandas as pd
from bokeh.io import output_notebook, show
from bokeh.plotting import figure, row
from bokeh.models import ColumnDataSource
from bokeh.palettes import viridis
from bokeh.models.annotations import Label
output_notebook()

def get_training_metrics(log_dir_path):
    # 构建日志文件路径
    log_file = os.path.join(log_dir_path, 'metrics.csv')

    # 从CSV文件中读取指标数据
    metrics = pd.read_csv(log_file, sep=',')

    # 从训练指标中选择'epoch'和'train_loss'列
    train_metrics = metrics[~metrics['train_loss'].isna()][['epoch', 'train_loss']]

    # 将'train_metrics'中的'epoch'减1，可能是为了与其他指标对齐
    train_metrics['epoch'] -= 1

    # 从验证指标中选择'val_loss'、'eff'、'pur'、'current_lr'和'epoch'列
    val_metrics = metrics[~metrics['val_loss'].isna()][['val_loss', 'eff', 'pur', 'current_lr', 'epoch']]

    # 合并训练和验证指标，使用'epoch'列作为连接键
    metrics = pd.merge(left=train_metrics, right=val_metrics, how='inner', on='epoch')

    return metrics

def plot_training_metrics(metrics):

    p1 = figure(title='Training validation loss', x_axis_label='Epoch', y_axis_label='Loss', y_axis_type="log")

    source = ColumnDataSource(metrics)

    cmap = viridis(3)

    for idx, y in enumerate(['train_loss', 'val_loss']):
        p1.circle(y=y, x='epoch', source=source, color=cmap[idx], legend_label=y)
        p1.line(x='epoch', y=y, source=source, color=cmap[idx], legend_label=y)


    p2 = figure(title='Purity on validation set', x_axis_label='Epoch', y_axis_label='Purity')
    p2.circle(y='pur', x='epoch', source=source, color=cmap[0], legend_label='Purity')
    p2.line(x='epoch', y='pur', source=source, color=cmap[0], legend_label='Purity')

    p3 = figure(title='Efficiency on validation set', x_axis_label='Epoch', y_axis_label='Efficiency')
    p3.circle(y='eff', x='epoch', source=source, color=cmap[0], legend_label='Efficiency')
    p3.line(x='epoch', y='eff', source=source, color=cmap[0], legend_label='Efficiency')

    show(row([p1,p2, p3]))

gnn_training = get_training_metrics("artifacts/Lambda500_pre_selection_quirk/gnn/quirk/version_7")
metric_training = get_training_metrics("artifacts/Lambda500_pre_selection_quirk/metric_learning/quirk/version_10")

#print(gnn_metrics)
print("gnn_pre_quirk_1000_1000:")
plot_training_metrics(gnn_training)
print("metric_pre_quirk_1000_1000:")
plot_training_metrics(metric_training)

Loading BokehJS ...

gnn_pre_quirk_1000_1000:


metric_pre_quirk_1000_1000:
